In [ ]:
import concurrent.futures
import time
# Define the local path where you want to save the file

bed_path = '/home/jupyter/top_aims_hg38.bed'
#for i in range(df_manifest.shape[0])
cnt = 0 
for i in range(4):
    a = time.time()
    # Define the path to your file in the GCS bucket
    vcf_gcs_path = df_manifest.loc[i]['vcf_file_location']
    local_vcf_path = vcf_gcs_path.split('/')[-1]
    ID = local_vcf_path.split('-')[0]
    # Use gsutil to copy the file from GCS to the local environment
    os.system(f'gsutil cp {vcf_gcs_path} {local_vcf_path}')
    os.system(f'gsutil cp {vcf_gcs_path}.tbi {local_vcf_path}.tbi')
    #vcf_file = pysam.VariantFile(vcf_path)
    os.system(f'bedtools intersect -a {local_vcf_path} -b {bed_path} > /home/jupyter/{ID}.vcf')
    os.system(f'rm {local_vcf_path}; rm {local_vcf_path}.tbi')
    cnt +=1
    print(time.time()-a)

In [ ]:
!cat /home/jupyter/*vcf > /home/jupyter/Merged.vcf

In [ ]:

#Import the vcf file
vcf = pd.read_csv('/home/jupyter/Merged.vcf', sep ='\t', header = None)
vcf.columns = list(header.loc[0])
vcf = vcf[vcf['FILTER']=='.']
#remove multiallelic? Or just set it regular 
vcf = vcf[~vcf['ALT'].str.contains(',')]
vcf = vcf[~vcf['INFO'].str.contains('AC=0')]
#vcf = vcf[~vcf['INFO'].str.contains('AC=1;')]
#vcf = vcf[~vcf['#CHROM'].str.contains('chrY')]
vcf = vcf[vcf['REF'].apply(len)==1]
import pandas as pd

# Assuming vcf and bed_file are your DataFrames
vcf.rename(columns={'#CHROM': 'Chr', 'POS': 'Start'}, inplace=True)


# Rename columns for clarity
bed_file.columns = ['Chr', 'Start', 'End', 'Ancestry_NucleotideChange', 'Score', 'Strand']

# Merge on 'Chr' and 'Start' columns
merged_df = pd.merge(vcf, bed_file, how='inner', on=['Chr', 'Start'])

# Extract the Ref and Alt from the Ancestry_NucleotideChange column in bed_file
merged_df[['Ancestry', 'Nucleotide_Change']] = merged_df['Ancestry_NucleotideChange'].str.split('_', expand=True)
merged_df[['Ref_bed', 'Alt_bed']] = merged_df['Nucleotide_Change'].str.split('>', expand=True)

# Filter to keep only rows where the REF and ALT in vcf match the ones in bed_file
filtered_vcf = merged_df[(merged_df['REF'] == merged_df['Ref_bed']) & (merged_df['ALT'] == merged_df['Alt_bed'])]

#Save Ancestry
ancestry = filtered_vcf['Ancestry'] 

# Drop the extra columns used for filtering
filtered_vcf = filtered_vcf.drop(columns=['End', 'Ancestry_NucleotideChange', 'Score', 'Strand', 'Ancestry', 'Nucleotide_Change', 'Ref_bed', 'Alt_bed'])

# Resulting filtered_vcf DataFrame contains only the desired variants
#print(filtered_vcf)
#split chrx and all other chromosomes 
vcfX = filtered_vcf[filtered_vcf['Chr'].str.contains('chrX')]
vcf = filtered_vcf[~filtered_vcf['Chr'].str.contains('chrX')]
# Define a function to substitute genotype values
def substitute_genotype(value):
    if value.startswith('1/1'):
        return 2
    elif value.startswith('0/1') or value.startswith('1/0'):
        return 1
    else:
        return 0

    
def substitute_genotype_chromX(value):
    if value.startswith('1/1'):
        return 1
    elif value.startswith('0/1') or value.startswith('1/0'):
        return 1
    else:
        return 0
# Apply the function to each column in the genomic info columns
vcf[vcf.columns[9:]] = vcf[vcf.columns[9:]].applymap(substitute_genotype)

# Display the modified DataFrame
#print(vcf.head())

vcfX[vcfX.columns[9:]] = vcfX[vcfX.columns[9:]].applymap(substitute_genotype_chromX)

vcf = vcf[vcf.columns[9:]]
vcfX = vcfX[vcfX.columns[9:]]
master_df = pd.concat([vcf,vcfX])
import pandas as pd

# Assuming master_df is your DataFrame with sample names as columns
# And ancestry is a pandas Series where each entry corresponds to the ancestry of the row in master_df

# Group by ancestry and sum the values for each sample
summed_df = master_df.groupby(ancestry).sum()

# Transpose the DataFrame so that rows become samples and columns become ancestries
summed_df = summed_df.T

# Display the resulting DataFrame
#print(summed_df)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score
import numpy as np
keep_index = vcf[vcf.columns[0:5]]
keep_index_X = vcfX[vcfX.columns[0:5]]
variant_df = pd.concat([keep_index,keep_index_X])
vcf = vcf[vcf.columns[9:]]
vcfX = vcfX[vcfX.columns[9:]]
train_df = pd.concat([vcf,vcfX])
copy_train_df = train_df.copy()
# 1. Create ethnicity labels for the samples
# Create a dictionary with the ethnicity labels
ethnicity_map = {sample: 'AFR' for sample in AFR}
#ethnicity_map.update({sample: 'ASJ' for sample in ASJ})
ethnicity_map.update({sample: 'EAS' for sample in EAS})
ethnicity_map.update({sample: 'AMR' for sample in AMR})
ethnicity_map.update({sample: 'SAS' for sample in SAS})
ethnicity_map.update({sample: 'EUR' for sample in EUR})

train_df = train_df[ethnicity_map.keys()]

# Create the labels (y) for each sample
samples = train_df.columns
y = np.array([ethnicity_map[sample] for sample in samples])

# 2. Transpose the train_df so that samples are rows and variants are columns
X = train_df.T  # Now each row is a sample and each column is a variant

# 3. Feature selection: Remove low-variance features (variants)
selector = VarianceThreshold(threshold=0.4)  # You can adjust the threshold
X_selected = selector.fit_transform(X)

# 4. Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)
Random Forest
# 5. Train a random forest classifier
clf = RandomForestClassifier(n_estimators=75, random_state=41)
clf.fit(X_train, y_train)

# 6. Make predictions on the test set
y_pred = clf.predict(X_test)

# 7. Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy}")
Model accuracy: 0.9597701149425287
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))

# Create a DataFrame for easier visualization
cm_df = pd.DataFrame(cm, index=np.unique(y_test), columns=np.unique(y_test))

# Visualize the confusion matrix using a heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(cm_df, annot=True, fmt='g', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Ethnicity')
plt.xlabel('Predicted Ethnicity')
plt.show()

# Display classification report
print(classification_report(y_test, y_pred, target_names=np.unique(y_test)))

In [ ]:
from sklearn.svm import SVC

svc = SVC(kernel='rbf', random_state=42)
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_svc)
print(f"Model accuracy: {accuracy}")
Model accuracy: 0.9770114942528736
clf.classes_
array(['AFR', 'AMR', 'EAS', 'EUR', 'SAS'], dtype='<U3')
y_predictions = []
# Assuming y_test contains the true ethnicity labels for the test samples
# y_test should be aligned with the indices of X_test

# Train an SVM model with probability=True to enable probability predictions
svc = SVC(kernel='rbf', probability=True, random_state=42)
svc.fit(X_train, y_train)

# Predict probabilities
y_proba = svc.predict_proba(X_test)


# Predict probabilities using Random Forest


# Iterate through each sample's probabilities and show the true ethnicity label
for i, probs in enumerate(y_proba):
    sorted_indices = probs.argsort()[::-1]  # Sort in descending order
    top_1 = sorted_indices[0]  # Index of the primary ethnicity
    top_2 = sorted_indices[1]  # Index of the second highest ethnicity (admixture likelihood)
    
    # Get the true ethnicity from y_test
    true_ethnicity = y_test[i]
    
    # Apply the rule: If primary ethnicity is EUR and prob < 0.60, and second highest is ASJ with prob > 0.30
    if (clf.classes_[top_1] == 'EUR' and probs[top_1] < 0.60) and (clf.classes_[top_2] == 'ASJ' and probs[top_2] > 0.30):
        predicted_ethnicity = 'ASJ'
        print('FREEZE')
        print(f"Sample {i+1}: True Ethnicity: {true_ethnicity}")
        print(f"Rule applied: Changed predicted ethnicity from EUR to ASJ")
    else:
        predicted_ethnicity = clf.classes_[top_1]
        print(f"Sample {i+1}: True Ethnicity: {true_ethnicity}")
        print(f"Primary Predicted Ethnicity: {predicted_ethnicity} with probability {probs[top_1]:.4f}")

    print(f"Possible Admixture: {clf.classes_[top_2]} with probability {probs[top_2]:.4f}")
    print("-" * 30)
    y_predictions.append(predicted_ethnicity)

In [ ]:
accuracy = accuracy_score(y_test, y_predictions)
print(f"Model accuracy: {accuracy}")
Model accuracy: 1.0
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Compute the confusion matrix
cm = confusion_matrix(y_test, y_predictions, labels=np.unique(y_test))

# Create a DataFrame for easier visualization
cm_df = pd.DataFrame(cm, index=np.unique(y_test), columns=np.unique(y_test))

# Visualize the confusion matrix using a heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(cm_df, annot=True, fmt='g', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Ethnicity')
plt.xlabel('Predicted Ethnicity')
plt.show()

# Display classification report
#print(classification_report(y_test, y_pred, target_names=np.unique(y_test)))

In [ ]:
import joblib

# Assuming X_selected is the transformed feature matrix after selection
# and variant_df contains information about the variants (with rows corresponding to variants in train_df)

# Get the indices of selected variants
selected_variant_indices = selector.get_support(indices=True)

# Extract the corresponding rows from variant_df based on the selected indices
selected_variants = variant_df.iloc[selected_variant_indices]

# Save the selected variant indices for future reference
selected_variants_file = '/home/jupyter/new_selected_variants.pkl'
joblib.dump(selected_variant_indices, selected_variants_file)
print(f"Selected variant indices saved to {selected_variants_file}")

# Optionally, save the entire selected variants DataFrame
selected_variants_df_file = '/home/jupyter/new_selected_variants_df.pkl'
selected_variants.to_pickle(selected_variants_df_file)
print(f"Selected variant details saved to {selected_variants_df_file}")
Selected variant indices saved to /home/jupyter/new_selected_variants.pkl
Selected variant details saved to /home/jupyter/new_selected_variants_df.pkl
# Save the trained SVM model
svm_model_file = '/home/jupyter/new_svm_model.pkl'
joblib.dump(svc, svm_model_file)  # Assuming svc is your trained SVM model
print(f"SVM model saved to {svm_model_file}")